# 1.MNIST 一个很典型的学习 case

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
#关于 MNIST 见：
#https://www.cnblogs.com/weiyinfu/p/7881997.html
#https://zhuanlan.zhihu.com/p/57950035

DATA_DIR = '/Users/finnqi/Documents'
NUM_STEPS = 1000
MINIBATCH_SIZE = 100

#关于 one_hot 编码见：https://zhuanlan.zhihu.com/p/37471802
data = input_data.read_data_sets(DATA_DIR, one_hot=True)

#为什么叫张量？
#1.张量是数学概念，可以囊括标量、向量、多维数组，1*1 张量是标量 1*n 张量是向量 n*n 张量是矩阵 n*n*n 张量是一个多维数组

#关于计算图：tensorflow中创建一个张量，并不会立即添加到计算图中，只有把张量赋值给变量或占位符，才会把张量添加到计算图中
#计算图的根本特征是能够识别出图中每个节点的依赖关系，这样不仅能够将计算分布到可用计算资源上，还能够避免执行不相关子集的冗余计算，
#最终能够得到更快更高效的计算方法

#x是占位符，用以传递数据对象
#W是变量，用以给定模型参数，与占位符不同的是，每次运行会话时，占位符会重新填充数据，而变量在图中保持固定状态，即各次迭代公用
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))

y_true = tf.placeholder(tf.float32, [None, 10])
y_pred = tf.matmul(x, W)

#about reduce_mean：Computes the mean of elements across dimensions of a tensor.
#https://www.tensorflow.org/api_docs/python/tf/math/reduce_mean
#交叉熵：cross_entropy
#参数为什么叫 logits?
#logit本身就是是一种函数，它把某个概率p从[0,1]映射到[-inf,+inf]（即正负无穷区间）。这个函数的形式化描述为：logit=ln(p/(1-p))。
#我们可以把logist理解为原生态的、未经缩放的，可视为一种未归一化的log 概率，如是[4, 1, -2]
#于是，Softmax的工作则是，它把一个系列数从[-inf, +inf] 映射到[0,1]，除此之外，它还把所有参与映射的值累计之和等于1，变成诸如[0.95, 0.05, 0]的概率向量。
#这样一来，经过Softmax加工的数据可以当做概率来用。
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=y_pred, labels=y_true))

gd_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

correct_mask = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y_true, 1))
accuracy = tf.reduce_mean(tf.cast(correct_mask, tf.float32))

#使用 with 有什么好处？
#1.当使用多个图时，通过使用 g=tf.Graph() with g.as_default(): 可以方便地切换默认图
#2.使用 with 可以启动会话后，能够不需要显式关闭，从而方便异常处理
with tf.Session() as sess:
    # Train
    #千万不要忘记初始化，该操作为变量分配内存并设置其初始化的值
    sess.run(tf.global_variables_initializer())
    for _ in range(NUM_STEPS):
        #这里使用的是随机梯度下降算法
        batch_xs, batch_ys = data.train.next_batch(MINIBATCH_SIZE)
        #注意这里占位符传入的是 x 和 y_true
        #因为 gd_step 用的 cross_entropy，cross_entropy 中使用 y_pred，而 y_pred 中使用了 x 
        #1.可见 tensorflow 中，字典结构中，要把最初始的数据传给占位符
        #2.数据传给占位符后，x 中的数据被完全覆盖了，和上次迭代没有任何关系，这是占位符和变量的一个主要区别
        sess.run(gd_step, feed_dict={x: batch_xs, y_true: batch_ys})

    # Test
    ans = sess.run(accuracy, feed_dict={x: data.test.images, y_true: data.test.labels})

print("Accuracy: {:.4}%".format(ans*100))

/Users/finnqi/anaconda2/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/finnqi/anaconda2/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/finnqi/anaconda2/envs/py3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/finnqi/anaconda2/envs/py3/lib/python3.7/site-

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/finnqi/Documents/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/finnqi/Documents/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/finnqi/Documents/t10k-images-idx3-ubyte.gz
Extracting /Users/finnqi/Documents/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Accuracy: 91.47%
